# Land Surface Temperature Analysis and Urban Heat Island Effect in NYC
<br> Radley Ciego </br>
<br> December 20, 2022 </br>
<br> GTECH GTECH 78518: Environmental Data Science </br>

<b> Research Question: </b> During a heatwave, which borough in New York City has the highest land surface temperature (LST)? Which borough has the lowest? 

### Data Sources:
<br> NYC Borough Boundaries (NYC OpenData) - .shp </br>
<br> - For clipping satellite data to NYC, processing, and basemap for GOES imagery </br>
<br> Land Surface Temperature (NASA/NOAA) - .nc </br>
<br> - Contains temporal LST data for analysis </br>

### GOES-16 Satellite
<br> Geostationary Operational Environmental Satellite (GOES) </br>
<br> Operated by NASA & the National Oceanic and Atmospheric Administration (NOAA) </br>
<br> Network Common Data Form (netCDF) file format: an array-oriented scientific datafile that are machine independent </br>

### Methodology:
<br> Utilizing matplotlib to plot figure with basemap and shapefile, colorfill by borough </br>
<br> Numpy to identify parellels and meridians for visualization </br>
<br> Process netCDF data using netCDF4 library </br>
<br> Return loops to run through each .nc file in directory </br>

In [1]:
from netCDF4 import Dataset
import os
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.use('TkAgg')
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.patches import PathPatch
import numpy as np

In [2]:
fig1 = plt.figure()
m1 = Basemap()
shp = m1.readshapefile('/Users/radleyciego/GitHub/environmental-data-science/data/Borough Boundaries/geo_export_dc3b4cc3-49b1-4394-816c-a0b738108e11', 'nyc_shapefile')
plt.close(fig1)
m1 = ()

In [3]:
# Plot shapefile, identify parallels and meridians for visualization
bbox = [shp[2][0],shp[2][1],shp[3][0],shp[3][1]]
fig,ax = plt.subplots(figsize=(14,8))
m = Basemap(llcrnrlon=bbox[0], llcrnrlat=bbox[1],urcrnrlon=bbox[2],
            urcrnrlat=bbox[3], projection='cyl')
m.readshapefile('/Users/radleyciego/GitHub/environmental-data-science/data/Borough Boundaries/geo_export_dc3b4cc3-49b1-4394-816c-a0b738108e11', 'nyc_shapefile', zorder=0)

parallels = np.linspace(bbox[1],bbox[3],5) # latitudes
m.drawparallels(parallels,labels=[True,False,False,False],fontsize=12)
meridians = np.linspace(bbox[0],bbox[2],5) # longitudes
m.drawmeridians(meridians,labels=[False,False,False,True],fontsize=12)

plt.show()

In [4]:
# Analyze shapefile for each borough
m.nyc_shapefile_info

[{'boro_code': 5.0,
  'boro_name': 'Staten Island',
  'shape_area': 1623631283.36,
  'shape_leng': 325924.002076,
  'RINGNUM': 1,
  'SHAPENUM': 1},
 {'boro_code': 5.0,
  'boro_name': 'Staten Island',
  'shape_area': 1623631283.36,
  'shape_leng': 325924.002076,
  'RINGNUM': 2,
  'SHAPENUM': 1},
 {'boro_code': 5.0,
  'boro_name': 'Staten Island',
  'shape_area': 1623631283.36,
  'shape_leng': 325924.002076,
  'RINGNUM': 3,
  'SHAPENUM': 1},
 {'boro_code': 5.0,
  'boro_name': 'Staten Island',
  'shape_area': 1623631283.36,
  'shape_leng': 325924.002076,
  'RINGNUM': 4,
  'SHAPENUM': 1},
 {'boro_code': 2.0,
  'boro_name': 'Bronx',
  'shape_area': 1187189499.3,
  'shape_leng': 463277.240478,
  'RINGNUM': 1,
  'SHAPENUM': 2},
 {'boro_code': 2.0,
  'boro_name': 'Bronx',
  'shape_area': 1187189499.3,
  'shape_leng': 463277.240478,
  'RINGNUM': 2,
  'SHAPENUM': 2},
 {'boro_code': 2.0,
  'boro_name': 'Bronx',
  'shape_area': 1187189499.3,
  'shape_leng': 463277.240478,
  'RINGNUM': 3,
  'SHAPEN

In [5]:
# Obtain latitude, longitude, and projection from nc data 

def lat_lon_reproj(nc_folder):
    os.chdir(nc_folder)
    full_direc = os.listdir()
    nc_files = [ii for ii in full_direc if ii.endswith('.nc')]
    g16_data_file = nc_files[0] # selct .nc file
    print(nc_files[0]) # print file name

    # designate dataset
    g16nc = Dataset(g16_data_file, 'r')
    var_names = [ii for ii in g16nc.variables]
    var_name = var_names[0]

    # GOES-R projection info and retrieving relevant constants
    proj_info = g16nc.variables['goes_imager_projection']
    lon_origin = proj_info.longitude_of_projection_origin
    H = proj_info.perspective_point_height+proj_info.semi_major_axis
    r_eq = proj_info.semi_major_axis
    r_pol = proj_info.semi_minor_axis

    # grid info
    lat_rad_1d = g16nc.variables['x'][:]
    lon_rad_1d = g16nc.variables['y'][:]
    
    # close file when finished
    g16nc.close()
    g16nc = None

    # create meshgrid filled with radian angles
    lat_rad,lon_rad = np.meshgrid(lat_rad_1d,lon_rad_1d)

    # lat/lon calc routine from satellite radian angle vectors

    lambda_0 = (lon_origin*np.pi)/180.0

    a_var = np.power(np.sin(lat_rad),2.0) + (np.power(np.cos(lat_rad),2.0)*\
                                             (np.power(np.cos(lon_rad),2.0)+\
                                              (((r_eq*r_eq)/(r_pol*r_pol))*\
                                               np.power(np.sin(lon_rad),2.0))))
    b_var = -2.0*H*np.cos(lat_rad)*np.cos(lon_rad)
    c_var = (H**2.0)-(r_eq**2.0)

    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)

    s_x = r_s*np.cos(lat_rad)*np.cos(lon_rad)
    s_y = - r_s*np.sin(lat_rad)
    s_z = r_s*np.cos(lat_rad)*np.sin(lon_rad)

    # latitude and longitude projection for plotting data on traditional lat/lon maps
    lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*\
                                   ((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)

    os.chdir('../')

    return lon,lat

In [6]:
# Obtain temporal and temperature data from each .nc file

def data_grab(nc_folder,nc_indx):
    os.chdir(nc_folder)
    full_direc = os.listdir()
    nc_files = [ii for ii in full_direc if ii.endswith('.nc')]
    g16_data_file = nc_files[nc_indx] # select .nc file
    print(nc_files[nc_indx]) # print file name

    # designate dataset
    g16nc = Dataset(g16_data_file, 'r')
    var_names = [ii for ii in g16nc.variables]
    var_name = var_names[0]

    # data info    
    data = g16nc.variables[var_name][:]
    data_units = g16nc.variables[var_name].units
    data_time_grab = ((g16nc.time_coverage_end).replace('T',' ')).replace('Z','')
    data_long_name = g16nc.variables[var_name].long_name
    
    # close file when finished
    g16nc.close()
    g16nc = None

    os.chdir('/Users/radleyciego/GitHub/environmental-data-science/data/001')
    # print test coordinates
    print('{} N, {} W'.format(lat[318,1849],abs(lon[318,1849])))

    return data,data_units,data_time_grab,data_long_name,var_name

In [7]:
# Create basemap for matplotlib plot
bbox = [shp[2][0],shp[2][1],shp[3][0],shp[3][1]]
fig,ax = plt.subplots(figsize=(14,8))
m = Basemap(llcrnrlon=bbox[0],llcrnrlat=bbox[1],urcrnrlon=bbox[2],
            urcrnrlat=bbox[3], projection='cyl')
m.readshapefile('/Users/radleyciego/GitHub/environmental-data-science/data/Borough Boundaries/geo_export_dc3b4cc3-49b1-4394-816c-a0b738108e11',
                'nyc_shapefile')
m.drawmapboundary(fill_color='#bdd5d5')
parallels = np.linspace(bbox[1],bbox[3],5) # latitudes
m.drawparallels(parallels,labels=[True,False,False,False],fontsize=12)
meridians = np.linspace(bbox[0],bbox[2],5) # longitudes
m.drawmeridians(meridians,labels=[False,False,False,True],fontsize=12)

{-74.25559136315213: ([<matplotlib.lines.Line2D at 0x7fd36c8a4c70>],
  [Text(-74.25503578085285, 40.49057816461902, '74.2556°W')]),
 -74.1166957883319: ([<matplotlib.lines.Line2D at 0x7fd36c8897f0>],
  [Text(-74.11614020603263, 40.49057816461902, '74.1167°W')]),
 -73.97780021351167: ([<matplotlib.lines.Line2D at 0x7fd36c889100>],
  [Text(-73.97724463121239, 40.49057816461902, '73.9778°W')]),
 -73.83890463869145: ([<matplotlib.lines.Line2D at 0x7fd36c3234c0>],
  [Text(-73.83834905639216, 40.49057816461902, '73.8389°W')]),
 -73.7000090638712: ([<matplotlib.lines.Line2D at 0x7fd36c3238e0>],
  [Text(-73.70056464617049, 40.49057816461902, '73.7°W')])}

In [8]:
# Retrieve borough code data for shading in by borough
boro_array = []
for ii in m.nyc_shapefile_info:
    boro_array.append(ii['boro_code'])

boro_unique = np.unique(boro_array)
colors = ['r','b','k','g','m']
for boro_iter in range(0,len(boro_unique)):
    patches   = []
    for ii,jj in zip(m.nyc_shapefile_info, m.nyc_shapefile):
        if ii['boro_code']==boro_unique[boro_iter]:
            patches.append(Polygon(np.array(jj),True))
        
    ax.add_collection(PatchCollection(patches, facecolor= colors[boro_iter],
                                      edgecolor='k', linewidths=1., zorder=2))

ax.add_collection(PatchCollection(patches, facecolor= colors[boro_iter], edgecolor='k', linewidths=1., zorder=2))

plt.show()

In [9]:
def lat_lon_reproj(nc_folder):
    os.chdir(nc_folder)
    full_direc = os.listdir()
    nc_files = [ii for ii in full_direc if ii.endswith('.nc')]
    g16_data_file = nc_files[0] # select .nc file
    print(nc_files[0]) # print file name

    # designate dataset
    g16nc = Dataset(g16_data_file, 'r')
    var_names = [ii for ii in g16nc.variables]
    var_name = var_names[0]

    # GOES-R projection info and retrieving relevant constants
    proj_info = g16nc.variables['goes_imager_projection']
    lon_origin = proj_info.longitude_of_projection_origin
    H = proj_info.perspective_point_height+proj_info.semi_major_axis
    r_eq = proj_info.semi_major_axis
    r_pol = proj_info.semi_minor_axis

    # grid info
    lat_rad_1d = g16nc.variables['x'][:]
    lon_rad_1d = g16nc.variables['y'][:]
    
    # close file when finished
    g16nc.close()
    g16nc = None

    # create meshgrid filled with radian angles
    lat_rad,lon_rad = np.meshgrid(lat_rad_1d,lon_rad_1d)

    # lat/lon calc routine from satellite radian angle vectors

    lambda_0 = (lon_origin*np.pi)/180.0

    a_var = np.power(np.sin(lat_rad),2.0) + (np.power(np.cos(lat_rad),2.0)*\
                                             (np.power(np.cos(lon_rad),2.0)+\
                                              (((r_eq*r_eq)/(r_pol*r_pol))*\
                                               np.power(np.sin(lon_rad),2.0))))
    b_var = -2.0*H*np.cos(lat_rad)*np.cos(lon_rad)
    c_var = (H**2.0)-(r_eq**2.0)

    r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)

    s_x = r_s*np.cos(lat_rad)*np.cos(lon_rad)
    s_y = - r_s*np.sin(lat_rad)
    s_z = r_s*np.cos(lat_rad)*np.sin(lon_rad)

    # latitude and longitude projection for plotting data on traditional lat/lon maps
    lat = (180.0/np.pi)*(np.arctan(((r_eq*r_eq)/(r_pol*r_pol))*\
                                   ((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))
    lon = (lambda_0 - np.arctan(s_y/(H-s_x)))*(180.0/np.pi)

    os.chdir('/Users/radleyciego/GitHub/environmental-data-science/data/001')

    return lon,lat


In [10]:
# Obtain temporal and LST data from each .nc file in directory using loop
def data_grab(nc_folder,nc_indx):
    os.chdir(nc_folder)
    full_direc = os.listdir()
    nc_files = [ii for ii in full_direc if ii.endswith('.nc')]
    g16_data_file = nc_files[nc_indx] # select .nc file
    print(nc_files[nc_indx]) # print file name

    # designate dataset
    g16nc = Dataset(g16_data_file, 'r')
    var_names = [ii for ii in g16nc.variables]
    var_name = var_names[0]

    # data info    
    data = g16nc.variables[var_name][:]
    data_units = g16nc.variables[var_name].units
    data_time_grab = ((g16nc.time_coverage_end).replace('T',' ')).replace('Z','')
    data_long_name = g16nc.variables[var_name].long_name
    
    # close file when finished
    g16nc.close()
    g16nc = None

    os.chdir('/Users/radleyciego/GitHub/environmental-data-science/data/001')
    # print test coordinates
    print('{} N, {} W'.format(lat[318,1849],abs(lon[318,1849])))

    return data,data_units,data_time_grab,data_long_name,var_name

In [11]:
# Create basemap for plottiing GOES-16 file

bbox = [shp[2][0],shp[2][1],shp[3][0],shp[3][1]]
fig,ax = plt.subplots(figsize=(14,8))
m = Basemap(llcrnrlon=bbox[0], llcrnrlat=bbox[1],urcrnrlon=bbox[2],
            urcrnrlat=bbox[3], projection='cyl')
m.readshapefile('/Users/radleyciego/GitHub/environmental-data-science/data/Borough Boundaries/geo_export_dc3b4cc3-49b1-4394-816c-a0b738108e11', 'nyc_shapefile', zorder=0)
m.drawmapboundary(fill_color='#bdd5d5')
parallels = np.linspace(bbox[1],bbox[3],5) # latitudes
m.drawparallels(parallels,labels=[True,False,False,False],fontsize=12)
meridians = np.linspace(bbox[0],bbox[2],5) # longitudes
m.drawmeridians(meridians,labels=[False,False,False,True],fontsize=12)

{-74.25559136315213: ([<matplotlib.lines.Line2D at 0x7fd36cc96eb0>],
  [Text(-74.25503578085285, 40.49057816461902, '74.2556°W')]),
 -74.1166957883319: ([<matplotlib.lines.Line2D at 0x7fd36cc94f40>],
  [Text(-74.11614020603263, 40.49057816461902, '74.1167°W')]),
 -73.97780021351167: ([<matplotlib.lines.Line2D at 0x7fd36d36a250>],
  [Text(-73.97724463121239, 40.49057816461902, '73.9778°W')]),
 -73.83890463869145: ([<matplotlib.lines.Line2D at 0x7fd36d36a520>],
  [Text(-73.83834905639216, 40.49057816461902, '73.8389°W')]),
 -73.7000090638712: ([<matplotlib.lines.Line2D at 0x7fd36d36a7f0>],
  [Text(-73.70056464617049, 40.49057816461902, '73.7°W')])}

In [12]:
# Finding boroughs and isolating them for shading in colors

boro_array = []
for ii in m.nyc_shapefile_info:
    boro_array.append(ii['boro_code'])

boro_unique = np.unique(boro_array)
colors = ['r','b','k','g','m']
for boro_iter in range(0,len(boro_unique)):
    patches   = []
    for ii,jj in zip(m.nyc_shapefile_info, m.nyc_shapefile):
        if ii['boro_code']==boro_unique[boro_iter]:
            patches.append(Polygon(np.array(jj),True))
        
    ax.add_collection(PatchCollection(patches, facecolor= colors[boro_iter],
                                      edgecolor='k', linewidths=1., zorder=2))

In [13]:
# GOES-16 LST section

nc_folder = '/Users/radleyciego/GitHub/environmental-data-science/data/001' # define folder where .nc files are located
lon,lat = lat_lon_reproj(nc_folder)

file_indx = 10 

data,data_units,data_time_grab,data_long_name,var_name = data_grab(nc_folder,file_indx)


OR_ABI-L2-LSTC-M6_G16_s20222210301171_e20222210303544_c20222210305137.nc


/var/folders/ff/9n9lg70d5d79jcc1pfmzhbsc0000gn/T/ipykernel_51160/4139826566.py:42: RuntimeWarning: invalid value encountered in sqrt
  r_s = (-1.0*b_var - np.sqrt((b_var**2)-(4.0*a_var*c_var)))/(2.0*a_var)
/var/folders/ff/9n9lg70d5d79jcc1pfmzhbsc0000gn/T/ipykernel_51160/4139826566.py:50: RuntimeWarning: invalid value encountered in sqrt
  ((s_z/np.sqrt(((H-s_x)*(H-s_x))+(s_y*s_y))))))


OR_ABI-L2-LSTC-M6_G16_s20222211601171_e20222211603544_c20222211605189.nc
40.702117919921875 N, 74.01709747314453 W


In [14]:
# clipping lat/lon/data vectors to shapefile bounds

# lower-left corner
llcrnr_loc_x = np.ma.argmin(np.ma.min(np.abs(np.ma.subtract(lon,bbox[0]))+\
                                      np.abs(np.ma.subtract(lat,bbox[1])),0))
llcrnr_loc = (np.ma.argmin(np.abs(np.ma.subtract(lon,bbox[0]))+\
                           np.abs(np.ma.subtract(lat,bbox[1])),0))[llcrnr_loc_x]
# upper-left corner
ulcrnr_loc_x = np.ma.argmin(np.ma.min(np.abs(np.ma.subtract(lon,bbox[0]))+\
                                      np.abs(np.ma.subtract(lat,bbox[3])),0))
ulcrnr_loc = (np.ma.argmin(np.abs(np.ma.subtract(lon,bbox[0]))+\
                           np.abs(np.ma.subtract(lat,bbox[3])),0))[ulcrnr_loc_x]
# lower-right corner
lrcrnr_loc_x = np.ma.argmin(np.ma.min(np.abs(np.ma.subtract(lon,bbox[2]))+\
                                      np.abs(np.ma.subtract(lat,bbox[1])),0))
lrcrnr_loc = (np.ma.argmin(np.abs(np.ma.subtract(lon,bbox[2]))+\
                           np.abs(np.ma.subtract(lat,bbox[1])),0))[lrcrnr_loc_x]
# upper-right corner
urcrnr_loc_x = np.ma.argmin(np.ma.min(np.abs(np.ma.subtract(lon,bbox[2]))+\
                                      np.abs(np.ma.subtract(lat,bbox[3])),0))
urcrnr_loc = (np.ma.argmin(np.abs(np.ma.subtract(lon,bbox[2]))+\
                           np.abs(np.ma.subtract(lat,bbox[3])),0))[urcrnr_loc_x]

x_bounds = [llcrnr_loc_x,ulcrnr_loc_x,lrcrnr_loc_x,urcrnr_loc_x]
y_bounds = [llcrnr_loc,ulcrnr_loc,lrcrnr_loc,urcrnr_loc]

In [15]:
# setting bounds for new clipped lat/lon/data vectors
plot_bounds = [np.min(x_bounds),np.min(y_bounds),np.max(x_bounds)+1,np.max(y_bounds)+1]

# new clipped data vectors
lat_clip = lat[plot_bounds[1]:plot_bounds[3],plot_bounds[0]:plot_bounds[2]]
lon_clip = lon[plot_bounds[1]:plot_bounds[3],plot_bounds[0]:plot_bounds[2]]
dat_clip = data[plot_bounds[1]:plot_bounds[3],plot_bounds[0]:plot_bounds[2]]

# plot the new data only in the clipped region
m.pcolormesh(lon_clip, lat_clip, dat_clip, latlon=True,zorder=999,
             alpha=0.95,cmap='jet') # plotting actual LST data
cb = m.colorbar()

data_units = ((data_units.replace('-','^{-')).replace('1','1}')).replace('2','2}')
plt.rc('text', usetex=True)
plt.title('ABI L2+ Land Surface Temperature on 2022-08-09',fontsize=14)

plt.show()

/var/folders/ff/9n9lg70d5d79jcc1pfmzhbsc0000gn/T/ipykernel_51160/1556377198.py:10: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  m.pcolormesh(lon_clip, lat_clip, dat_clip, latlon=True,zorder=999,


### Results:
<br> Based on the netCDF4 overlayed above the NYC basemap, it can be concluded that Brooklyn and Queens are most impacted by the urban heat island effect during heatwaves, with clusters of LST at about 114 fahrenheit. Staten Island is the least impacted by urban heat island effect. Neighborhoods with the highest concentration of urban development and highways have disproportinately higher LST, while neighborhoods with waterfront access and higher concentrations of greenspace have lower LST. </br>